In [15]:
# ライブラリをインポート

# データ操作と数値計算のため
import pandas as pd
import numpy as np

# データ可視化のため
import matplotlib.pyplot as plt
import seaborn as sns

# 統計モデリングと計量経済分析のため
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

# for ignore warning
import warnings
warnings.filterwarnings('ignore')

In [3]:
url = "https://www.fbc.keio.ac.jp/~tyabu/keiryo/cons_data.csv"
df = pd.read_csv(url)
df.head()

,prefecture,income,cons,age,number,house
0,Hokkaido,37.0498,29.8903,49.0,1033,73.1
1,Aomori,34.0994,26.0726,49.6,334,76.2
2,Iwate,38.0284,31.5566,50.2,381,76.6
3,Miyagi,38.4490,31.8181,48.8,369,69.4
4,Akita,40.1957,29.2273,50.4,349,80.4


# 9.4.1節の推定結果

In [16]:
# ロバスト標準誤差を用いた通常のOLS推定
endog = df['cons']            # 被説明変数
exog = df['income']           # 説明変数
exog = sm.add_constant(exog)  # 定数項を追加

mod_ols = sm.OLS(
    endog, # 被説明変数
    exog   # 説明変数
)
res_ols = mod_ols.fit(cov_type = 'HC1') # ロバスト標準誤差を指定
print(res_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                   cons   R-squared:                       0.744
Model:                            OLS   Adj. R-squared:                  0.739
Method:                 Least Squares   F-statistic:                     115.7
Date:                Sat, 13 Jul 2024   Prob (F-statistic):           5.05e-14
Time:                        11:50:01   Log-Likelihood:                -69.146
No. Observations:                  47   AIC:                             142.3
Df Residuals:                      45   BIC:                             146.0
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.4900      2.218      2.927      0.0

In [21]:
# 通常の標準誤差を用いたWLS推定
mod_wls = sm.WLS(
    endog, # 被説明変数
    exog,  # 説明変数
    weights = df['number'] # 重みを指定
)
res_wls = mod_wls.fit()
print(res_wls.summary())

                            WLS Regression Results                            
Dep. Variable:                   cons   R-squared:                       0.766
Model:                            WLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     147.6
Date:                Sat, 13 Jul 2024   Prob (F-statistic):           8.36e-16
Time:                        11:52:11   Log-Likelihood:                -68.534
No. Observations:                  47   AIC:                             141.1
Df Residuals:                      45   BIC:                             144.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.4572      2.109      2.588      0.0

In [19]:
# ロバスト標準誤差を用いたWLS推定
mod_wls_robust = sm.WLS(
    endog, # 被説明変数
    exog,  # 説明変数
    weights = df['number'] # 重みを指定
)
res_wls_robust = mod_wls.fit(cov_type = 'HC1') # ロバスト標準誤差を指定
print(res_wls_robust.summary())

                            WLS Regression Results                            
Dep. Variable:                   cons   R-squared:                       0.766
Model:                            WLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     126.9
Date:                Sat, 13 Jul 2024   Prob (F-statistic):           1.10e-14
Time:                        11:52:05   Log-Likelihood:                -68.534
No. Observations:                  47   AIC:                             141.1
Df Residuals:                      45   BIC:                             144.8
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.4572      2.225      2.453      0.0

In [25]:
# 推定結果を1つの表にまとめる．
results_table = summary_col(
    results = [
        res_ols,
        res_wls,
        res_wls_robust
    ],
    model_names = [
        'ols',
        'wls',
        'wls(robust)'
    ],
    stars = True,
    float_format = '%0.3f',
    info_dict = {
        'N':lambda x: "{0:d}".format(int(x.nobs))
    }
)

print(results_table)


                 ols      wls    wls(robust)
--------------------------------------------
const          6.490*** 5.457**  5.457**    
               (2.218)  (2.109)  (2.225)    
income         0.610*** 0.642*** 0.642***   
               (0.057)  (0.053)  (0.057)    
R-squared      0.744    0.766    0.766      
R-squared Adj. 0.739    0.761    0.761      
N              47       47       47         
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01
